In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import numpy as np
np.bool = np.bool_

In [3]:
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

In [4]:
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [5]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [6]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .enableHiveSupport()
    .getOrCreate()
)

25/02/17 22:33:04 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
25/02/17 22:33:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-96573755-4012-4e3c-a2fe-f7979b08d256;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central
	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 217ms :: artifacts dl 5ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.google.errorprone#error_prone_annotations;2.10.0 from central in [default]
	io.delta#delta-iceberg_2.12;3.0.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 f

In [7]:
base_path = f"{hdfs_lakehouse_base_path}/bronze/trump_btc"

# List of folder names (from the structure provided)
folder_names = ["BTC", "Trump"]

# Dictionary to store DataFrames
dataframes = {}

# Loop through each folder and read the CSV file inside it
for folder_name in folder_names:
    # Construct the file path
    file_path = os.path.join(base_path, folder_name)
    
    # Read the CSV file into a DataFrame
    dataframes[folder_name] = (spark.read
                                        .option("header", "true")
                                        .option("inferSchema", "true")
                                        .csv(file_path))
    print(f"Loaded DataFrame for {folder_name}_raw")

# Example: Accessing one of the DataFrames
btc_df = dataframes["BTC"]
trump_df=dataframes["Trump"]


Loaded DataFrame for BTC_raw
Loaded DataFrame for Trump_raw


In [8]:
btc_df.toPandas().head()

,Date,Close,High,Low,Open,Volume
0,None,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
1,2014-09-17,457.3340148925781,468.17401123046875,452.4219970703125,465.864013671875,21056800
2,2014-09-18,424.44000244140625,456.8599853515625,413.10400390625,456.8599853515625,34483200
3,2014-09-19,394.7959899902344,427.8349914550781,384.5320129394531,424.1029968261719,37919700
4,2014-09-20,408.90399169921875,423.2959899902344,389.88299560546875,394.6730041503906,36863600


In [9]:
trump_df.toPandas().head()

,date,favorites,id,isRetweet,retweets,text
0,1730391364149,8601.0,1.134029e+17,False,2282.0,"""<p>I would like to wish our great Bitcoiners a Happy 16th Anniversary of Satoshi’s White Paper. We will end Kamala’s war on crypto, &amp; Bitcoin will be MADE IN THE USA! VOTE TRUMP! <a href=""""https://truthsocial.com/tags/Bitcoin"""" class=""""mention hashtag"""" rel=""""tag"""">#<span>Bitcoin</span></a> <a href=""""https://truthsocial.com/tags/FreeRossDayOne"""" class=""""mention hashtag"""" rel=""""tag"""">#<span>FreeRossDayOne</span></a></p>"""
1,1727709073460,8655.0,1.132271e+17,False,2516.0,"""<p>I promised to Make America Great Again, this time with crypto. <a href=""""https://truthsocial.com/tags/WorldLibertyFi"""" class=""""mention hashtag"""" rel=""""tag"""">#<span>WorldLibertyFi</span></a> is planning to help make America the crypto capital of the world! The whitelist for eligible persons is officially open – this is your chance to be part of this historic moment. Join: <a href=""""https://links.truthsocial.com/link/113073993409242797"""" rel=""""nofollow noopener noreferrer"""" target=""""_blank""""><span class=""""invisible"""">https://www.</span><span class="""""""">worldlibertyfinancial.com/</span><span class=""""invisible""""></span></a></p>"""
2,1727190126685,0.0,1.131931e+17,True,0.0,"""<p>RT <span class=""""h-card""""><a href=""""https://truthsocial.com/@realtrumpcoins"""" class=""""u-url mention"""">@<span>realtrumpcoins</span></a></span>The ONLY official coin designed by <span class=""""h-card""""><a href=""""https://truthsocial.com/@realDonaldTrump"""" class=""""u-url mention"""">@<span>realDonaldTrump</span></a></span>"
3,1726957374556,7797.0,1.131779e+17,False,2029.0,"""<p><a href=""""https://links.truthsocial.com/link/113161965355999676"""" rel=""""nofollow noopener noreferrer"""" target=""""_blank""""><span class=""""invisible"""">https://</span><span class=""""ellipsis"""">justthenews.com/politics-polic</span><span class=""""invisible"""">y/all-things-trump/trump-uses-bitcoin-buy-hamburgers-new-york</span></a></p>"""
4,1726954291754,7113.0,1.131777e+17,False,1909.0,"""<p>“Trump Credits Success of His NFTs for Change of Heart on Bitcoin and Crypto” <br/><a href=""""https://links.truthsocial.com/link/113177676465018997"""" rel=""""nofollow noopener noreferrer"""" target=""""_blank""""><span class=""""invisible"""">https://</span><span class=""""ellipsis"""">decrypt.co/249755/trump-credit</span><span class=""""invisible"""">s-nfts-change-of-heart-bitcoin-crypto</span></a></p>"""


In [10]:
# cleaning the btc data

from pyspark.sql.functions import col, to_date

btc_df_std = btc_df.withColumnRenamed("Date", "date") \
                      .withColumn("date", to_date(col("date"))) \
                      .dropna()  # Remove missing values

btc_df_std.show(5)

+----------+------------------+------------------+------------------+-----------------+--------+
|      date|             Close|              High|               Low|             Open|  Volume|
+----------+------------------+------------------+------------------+-----------------+--------+
|2014-09-17| 457.3340148925781|468.17401123046875| 452.4219970703125| 465.864013671875|21056800|
|2014-09-18|424.44000244140625| 456.8599853515625|   413.10400390625|456.8599853515625|34483200|
|2014-09-19| 394.7959899902344| 427.8349914550781| 384.5320129394531|424.1029968261719|37919700|
|2014-09-20|408.90399169921875| 423.2959899902344|389.88299560546875|394.6730041503906|36863600|
|2014-09-21| 398.8210144042969| 412.4259948730469| 393.1809997558594|408.0849914550781|26580100|
+----------+------------------+------------------+------------------+-----------------+--------+
only showing top 5 rows



In [11]:
spark.sql("CREATE SCHEMA IF NOT EXISTS trump_btc")

DataFrame[]

In [12]:
#saving btc data it in silver
(btc_df_std.write
            .format("delta")
            .mode("overwrite")
            .option("path",f"{hdfs_lakehouse_base_path}/silver/trump_btc/BTC/")
            .saveAsTable("trump_btc.BTC")
)


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, date_format,regexp_replace
from pyspark.sql.types import StringType, BooleanType, DoubleType, LongType, TimestampType
import re

trump_df = trump_df.withColumn("date", col("date").cast("bigInt"))
trump_df = trump_df.withColumn("date", date_format(from_unixtime(col("date") / 1000), "yyyy-MM-dd"))
trump_df = trump_df.withColumn("text", regexp_replace(col("text"), "<[^>]+>", ""))


In [14]:
trump_df_std=trump_df
trump_df_std.show()

+----------+---------+-------------------+---------+--------+--------------------+
|      date|favorites|                 id|isRetweet|retweets|                text|
+----------+---------+-------------------+---------+--------+--------------------+
|2024-10-31|     8601| 113402928440845044|    false|    2282|"I would like to ...|
|2024-09-30|     8655| 113227141838155766|    false|    2516|"I promised to Ma...|
|2024-09-24|        0| 113193132142396661|     true|       0|"RT @realtrumpcoi...|
|2024-09-22|     7797| 113177878498911997|    false|    2029|"https://justthen...|
|2024-09-21|     7113| 113177676464453229|    false|    1909|"“Trump Credits S...|
|2024-09-21|    13066| 113175718594981285|    false|    3787|"Good morning eve...|
|2024-09-18|    12602| 113160641426571582|    false|    3551|“President Trump ...|
|2024-09-12|     5400| 113125745300178317|    false|    1451|"https://www.coin...|
|2024-07-31|     9906| 112881568598723413|    false|    2533|"Just spoke at th...|
|202

In [15]:
#saving btc data it in silver
(trump_df_std.write
            .format("delta")
            .mode("overwrite")
            .option("path",f"{hdfs_lakehouse_base_path}/silver/trump_btc/trump/")
            .saveAsTable("trump_btc.trump")
)
